<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_T81_558/blob/master/Part2_4_Pandas_functional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part2.4 : Apply and Map
如果您曾经使用过大数据或函数式编程语言，则可能听说过map / reduce。 map和reduce是将您创建的函数应用于data frames的两个功能。 Pandas支持函数式编程技术，可让您跨整个data frames使用函数。 除了编写的功能外，Pandas还提供了一些用于data frames的标准功能。

## 1. 在Dataframes中使用Map
map函数允许将列中的某些值映射到其他值来转换列。考虑Auto-MPG数据集，该数据集包含一个字段origin_name，该字段名称包含一个在1到3之间的值，该值表示每辆车的地理原点。我们可以看到如何使用map函数将这个数字原点转换为每个原点的文本名称.

首先加载数据集

In [ ]:
import os 
import pandas as pd
import numpy as np

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 5)

display(df)

,mpg,cylinders,displacement,...,year,origin,name
0,18.0,8,307.0,...,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,...,70,1,buick skylark 320
...,...,...,...,...,...,...,...
396,28.0,4,120.0,...,82,1,ford ranger
397,31.0,4,119.0,...,82,1,chevy s-10


Pandas中的map函数是对单个列进行操作。可以为map提供一个字典，以转换目标列。map中字典的key指定要处理列中的值，value指定应该转换为这些key的值。

下面的代码显示了map函数如何将数字值1、2和3转换为北美、欧洲和亚洲的字符串值。

In [ ]:
# 对dataframes创建一个新列
df['origin_name'] = df['origin'].map({1:"North America",2:"Europe",3:'Asia'})

# 混洗数据
df = df.reindex(np.random.permutation(df.index))

# 显示处理后的数据
pd.set_option('display.max_columns',7)
pd.set_option('display.max_rows',10)

display(df)

,mpg,cylinders,displacement,...,origin,name,origin_name
77,22.0,4,121.0,...,2,volkswagen 411 (sw),Europe
350,34.7,4,105.0,...,1,plymouth horizon 4,North America
124,11.0,8,350.0,...,1,oldsmobile omega,North America
169,20.0,6,232.0,...,1,amc gremlin,North America
90,12.0,8,429.0,...,1,mercury marquis brougham,North America
...,...,...,...,...,...,...,...
321,32.2,4,108.0,...,3,toyota corolla,Asia
82,23.0,4,120.0,...,3,toyouta corona mark ii (sw),Asia
286,17.6,8,302.0,...,1,ford ltd landau,North America
69,12.0,8,350.0,...,1,oldsmobile delta 88 royale,North America


## 2. 在Dataframes中使用Apply
Dataframes的apply()可以在整个数据帧上运行一个函数。该函数可以使用传统的命名函数，也可以使用lambda函数。Python也将对Dataframes中的每一行或每一列执行提供的函数。函数的axis参数指定跨行或跨列运行即可。(eg:axis = 1，使用行)。

**注：**在Pandas中的函数的axis = 0是指行，而在Python、Numpy中是指列

下面的代码计算一个称为efficiency的级数，即displacement除以horsepower。

In [ ]:
# 在pandas中的函数是axis=1是列操作
efficiency = df.apply(lambda x: x['displacement']/x['horsepower'], axis=1)
display(efficiency[0:10])

77     1.592105
350    1.666667
124    1.944444
169    2.320000
90     2.166667
141    1.180723
79     1.391304
227    2.250000
306    1.504348
87     2.413793
dtype: float64

现在，可以将这个Serise作为新列或替换现有列插入到dataframes中。

下面的代码将这个新serise插入到dataframes中。

In [ ]:
df['efficiency'] = efficiency
display(df)

,mpg,cylinders,displacement,...,name,origin_name,efficiency
77,22.0,4,121.0,...,volkswagen 411 (sw),Europe,1.592105
350,34.7,4,105.0,...,plymouth horizon 4,North America,1.666667
124,11.0,8,350.0,...,oldsmobile omega,North America,1.944444
169,20.0,6,232.0,...,amc gremlin,North America,2.320000
90,12.0,8,429.0,...,mercury marquis brougham,North America,2.166667
...,...,...,...,...,...,...,...
321,32.2,4,108.0,...,toyota corolla,Asia,1.440000
82,23.0,4,120.0,...,toyouta corona mark ii (sw),Asia,1.237113
286,17.6,8,302.0,...,ford ltd landau,North America,2.341085
69,12.0,8,350.0,...,oldsmobile delta 88 royale,North America,2.187500


## 3. 在特征工程中使用Map、Apply
我们将看到如何使用map、apply和grouping计算复杂的特征。数据集是以下CSV:
* [https://www.irs.gov/pub/irs-soi/16zpallagi.csv](https://www.irs.gov/pub/irs-soi/16zpallagi.csv)

此URL包含“SOI税收统计-个人所得税统计”的美国政府公共数据。网站的入口点在这里
* [https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi](https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi)

描述该数据的文档在上面的链接中。


对于这些特征，我们将尝试估算每个邮政编码的调整后总收入（adjusted gross income -AGI）。 数据文件包含许多列。 但是，您将仅使用以下内容：

* STATE： 州
* zipcode：邮政编码
* agi_stub：六种不同的年收入等级(1至6)
* N1：每个agi存根的报税表数目

**注：**对于每个zipcode和每个agi_stub，将有6行。你可以用0或99999跳过邮政编码。

我们将用这些列创建一个输出CSV;但是，每个邮政编码只有一行。计算收入等级的加权平均值。例如，以下六行用于63017:


|zipcode |agi_stub | N1 |
|--|--|-- |
|63017     |1 | 4710 |
|63017     |2 | 2780 |
|63017     |3 | 2130 |
|63017     |4 | 2010 |
|63017     |5 | 5240 |
|63017     |6 | 3510 |


我们必须将这六行合并为一。 出于隐私原因，AGI分为6个存储桶。 我们需要组合各个存储桶，并估算邮政编码的实际AGI。 为此，请考虑N1的值：

* 1 = 1 to 25,000
* 2 = 25,000 to 50,000
* 3 = 50,000 to 75,000
* 4 = 75,000 to 100,000
* 5 = 100,000 to 200,000
* 6 = 200,000 or more

每个范围的中值大约为：

* 1 = 12,500
* 2 = 37,500
* 3 = 62,500 
* 4 = 87,500
* 5 = 112,500
* 6 = 212,500

使用它，您可以估计63017的平均AGI


```
>>> totalCount = 4710 + 2780 + 2130 + 2010 + 5240 + 3510
>>> totalAGI = 4710 * 12500 + 2780 * 37500 + 2130 * 62500 
    + 2010 * 87500 + 5240 * 112500 + 3510 * 212500
>>> print(totalAGI / totalCount)

88689.89205103042
```



In [ ]:
# 读取数据
import pandas as pd
df=pd.read_csv('https://www.irs.gov/pub/irs-soi/16zpallagi.csv')

首先，删除所有为0或99999的邮政编码。并且选择了我们需要的三个字段。

In [ ]:
# df.loc[]:通过标签或布尔数组访问一组行和列
df = df.loc[(df['zipcode']!=0) & (df['zipcode']!=99999),['STATE','zipcode','agi_stub','N1']]

pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 10)

display(df)

,STATE,zipcode,agi_stub,N1
6,AL,35004,1,1510
7,AL,35004,2,1410
8,AL,35004,3,950
9,AL,35004,4,650
10,AL,35004,5,630
...,...,...,...,...
179785,WY,83414,2,40
179786,WY,83414,3,40
179787,WY,83414,4,0
179788,WY,83414,5,40


我们用map函数将所有的agi_stub替换为正确的中值。

In [ ]:
medians = {1:12500,2:37500,3:62500,4:87500,5:112500,6:212500}
df['agi_stub']  = df['agi_stub'].map(medians)

pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 10)
display(df)

,STATE,zipcode,agi_stub,N1
6,AL,35004,12500,1510
7,AL,35004,37500,1410
8,AL,35004,62500,950
9,AL,35004,87500,650
10,AL,35004,112500,630
...,...,...,...,...
179785,WY,83414,37500,40
179786,WY,83414,62500,40
179787,WY,83414,87500,0
179788,WY,83414,112500,40


我们根据邮政编码对Datafranes进行分组。

In [ ]:
groups = df.groupby(by='zipcode')
print(groups)

使用apply(),并且使用lambda函数计算AGI估计。

In [ ]:
df = pd.DataFrame(groups.apply( 
    lambda x:sum(x['N1']*x['agi_stub'])/sum(x['N1']))) \
    .reset_index()

In [ ]:
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 10)

display(df)

,zipcode,0
0,1001,52895.322940
1,1002,64528.451001
2,1003,15441.176471
3,1005,54694.092827
4,1007,63654.353562
...,...,...
29867,99921,48042.168675
29868,99922,32954.545455
29869,99925,45639.534884
29870,99926,41136.363636


现在，我们可以重命名新的agi_estimate列。

In [ ]:
df.columns = ['zipcode','agi_estimate']

In [ ]:
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 10)

display(df)

,zipcode,agi_estimate
0,1001,52895.322940
1,1002,64528.451001
2,1003,15441.176471
3,1005,54694.092827
4,1007,63654.353562
...,...,...
29867,99921,48042.168675
29868,99922,32954.545455
29869,99925,45639.534884
29870,99926,41136.363636


最后，检查邮政编码63017的值是否正确。

In [ ]:
df[ df['zipcode']==63017 ]

,zipcode,agi_estimate
19909,63017,88689.892051
